In [ ]:
import json
import psycopg2
import inventoryCleaner as iC

def is_json(myjson):
  try:
    json_object = json.loads(myjson)
  except ValueError:
    return False
  return True

In [ ]:
shapefile = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RoadInventory.geojson'

In [ ]:
f = open(shapefile, 'r')

In [ ]:
f.readline()

In [ ]:
str1 = '{ "type": "Feature", "properties": { "RoadInvent": 4600, "CRN": "A4600", "RoadSegmen": 46, "FromMeasur": 0.000000, "ToMeasure": 0.080000, "AssignedLe": 0.080000, "Assigned_1": 1, "StreetList": 181961, "StreetName": "DUNCAN LANE", "StreetNa_1": null, "City": 55, "County": "A", "MunicipalS": 2, "FromEndTyp": 5, "FromStreet": "HARWICH TOWN LINE", "FromCity": 126, "FromState": 0, "ToEndType": 3, "ToStreetNa": "CUL_DE_SAC", "ToCity": 0, "ToState": 0, "MileageCou": 1, "RouteKey": "L181961 NB", "RouteFrom": 0.000000, "RouteTo": 0.000000, "EquationRo": 0.000000, "Equation_1": 0.000000, "RouteSyste": null, "RouteNumbe": null, "SubRoute": null, "RouteDirec": null, "RouteType": 0, "RouteQuali": 0, "RPA": "CCC", "MPO": "Cape Cod", "MassDOTHig": 5, "UrbanType": 1, "UrbanizedA": "05167", "Functional": 0, "FederalFun": 7, "Jurisdicti": "0", "TruckRoute": 0, "TruckNetwo": 0, "NHSStatus": 0, "MHS": 0, "FederalAid": null, "FacilityTy": 1, "StreetOper": 2, "AccessCont": 0, "TollRoad": 0, "NumberOfPe": 0, "FutureFaci": 0, "RightSidew": 0, "RightShoul": 2, "RightSho_1": 2, "MedianType": 0, "MedianWidt": 0, "LeftSidewa": 0, "LeftShould": 0, "UndividedL": 0, "Undivide_1": 0, "LeftShou_1": 0, "SurfaceTyp": 6, "SurfaceWid": 20, "RightOfWay": 30, "NumberOfTr": 2, "OppositeNu": 0, "Curbs": 0, "Terrain": 2, "SpeedLimit": 0, "OpposingDi": 0, "Structural": 1, "ADT": 0, "ADTStation": 0, "ADTDerivat": 0, "ADTYear": 0, "IRI": 0, "IRIYear": 0, "IRIStatus": 0, "PSI": 0.000000, "PSIYear": 0, "HPMSCode": 0, "HPMSSample": null, "AddedRoadT": 0, "DateActive": "2004\\/01\\/01", "LifecycleS": 3, "ITEM_ID": 23403, "Shape_len": 126.924548 }, "geometry": { "type": "LineString", "coordinates": [ [ 322703.615272112190723, 828715.10250592941884, -0.000150013482198 ], [ 322725.117224132292904, 828718.917417933000252, 0.013849986717105 ], [ 322734.48068014101591, 828727.934761941432953, 0.021849986631423 ], [ 322726.157608133275062, 828763.308841974241659, 0.044849986676127 ], [ 322714.357032122206874, 828817.792810025042854, 0.079849986592308 ] ] } }'

In [ ]:
is_json(str1)

In [ ]:
js = json.loads(str1)

In [ ]:
js["properties"]

In [ ]:
colnames = js["properties"]
for n in colnames:
    print(colnames[n])

In [ ]:
con = psycopg2.connect(database='roadInventory', user='louisf') 
cur = con.cursor()


cur.execute('ALTER TABLE segments ADD "Structural" INT')          
con.commit()
if con:
    con.close()

In [ ]:
con = psycopg2.connect(database='roadInventory', user='louisf') 
cur = con.cursor()
cur.execute("Select * FROM segments")
colnames = [desc[0] for desc in cur.description]
con.close()
colnames

In [ ]:
con = psycopg2.connect(database='roadInventory', user='louisf') 
cur = con.cursor()
try:
    cur.execute("INSERT INTO segments VALUES(999,999,'TEST ROAD', 65, 0, 3)")
    con.commit()
except e:
    if con:
        con.rollback()
    print(e)

if con:
    con.close()

In [ ]:
con = psycopg2.connect(database='roadInventory', user='louisf') 
cur = con.cursor()
cur.execute("select * from segments")

In [2]:
import json
import psycopg2
import psycopg2.extras
import inventoryCleaner as iC

def is_json(myjson):
  try:
    json_object = json.loads(myjson)
  except ValueError:
    return False
  return True

In [3]:
inv = iC.Inventory("roadInventory", "segments")

Connected to database roadInventory


In [4]:
inv.parseGeoJson("/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RoadInventory.geojson")

1

In [ ]:
l = ['RoadInvent', 'CRN', 'StreetName', 'SpeedLimit', 'Terrain', 'Structural', 'AssignedLe', 'RouteDirec', 'RightShoul', 'RightSho_1', 'MedianType', 'MedianWidt', 'RoadSegmen', 'StreetList']

In [ ]:
inv.entries[0]

In [ ]:
help(iC.insertIntoDB)

In [ ]:
iC.insertIntoDB("roadInventory", "segments", inv.entries[0])

In [ ]:
tuple(l)

In [ ]:
con = psycopg2.connect(database='roadInventory', user='louisf') 
cur = con.cursor()
l = list(inv.entries[0].keys())
l2 = ', '.join(l)
cols = tuple(l)
dispcols = l2
vals = [inv.entries[0][x] for x in cols]
vals_str_list = ["%s"] * len(vals)
cur.execute("SELECT * FROM segments")
vals_str = ", ".join(vals_str_list)
statement = "INSERT INTO " + 'segments ' + "({dispcols}) VALUES ({vals_str})".format(
    dispcols=dispcols, vals_str=vals_str)
print(statement)
cur.execute("INSERT INTO " + 'segments ' + "({dispcols}) VALUES ({vals_str})".format(
    dispcols=dispcols, vals_str=vals_str), vals)
con.commit()

In [ ]:
for segs in inv.entries:
    iC.insertIntoDB("roadInventory", "segments", segs)

Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadInventory
Connected to database roadIn

In [ ]:
entry = {'RoadInvent':0}
con = psycopg2.connect(database='roadInventory', user='louisf') 
cur = con.cursor()
cols = entry.keys()
vals = [entry[x] for x in cols]
vals_str_list = ["%s"] * len(vals)
vals_str = ", ".join(vals_str_list)
statement = "INSERT INTO " + 'segments ' + "({cols}) VALUES({vals_str})".format(
    cols=cols, vals_str=vals_str)
print(statement)
cur.execute(statement, vals)

In [ ]:
song = dict()
song['title'] = 'song 1'
song['artist'] = 'artist 1'

cols = song.keys()
vals = [song[x] for x in cols]
vals_str_list = ["%s"] * len(vals)
vals_str = ", ".join(vals_str_list)
s="INSERT INTO song_table ({cols}) VALUES ({vals_str})".format(
               cols=cols, vals_str=vals_str)
s

In [ ]:
d = {"A":1, "b":2}

In [ ]:
dict((k.lower() if isinstance(k, str) else k, v.lower() if isinstance(v, str) else v) for k,v in d.items())